# Simulation of Urban Agriculture



In the following exercise, we will create a simulation of an efficient trade model for self-produced goods. The model will consider consumer preferences and calculate the social utility of these goods based on all individual preferences. This involves calculating the price of the social utility for each good and the total production. A monetary base is then computed, and a percentage of it is distributed to individuals using the same parameters, taking into account their individual production. Finally, the model simulates various production scenarios with the aim of minimizing the total remaining balance for all individuals.

## Individuals

Each individual is has two inmutable variables in the model: **preferences matrix** and **production space**. Although this two can change in real life for the moment they will stay inmutable. 


### Preferences Matrix

Given $k$ preferences over $n$ goods, we will construct a preference matrix with dimensions $n \times k$. To simplify calculations, we will work with a square matrix by taking $n$ preferences. Each column of the preference matrix represents a good, and each row represents a normalized basket of preferences. We assume that all these preference baskets provide the same utility to the individual and are calculated on a 100-point preference system (assigning a value out of 100 points to each good). It's important to note that this preference matrix does not assume rationality among preferences and offers a different approach to understanding market dynamics.

### Production Space

Additionally,the production space represents the amount of space each individual can allocate for production. We will assume that each individual's production space is fully utilized for each term.


On the other hand, all individuals have three time-dependent variables: a **money balance** and a **production vector**, and a space occupation vector.

### Money Balance

The money balance begans in $0$ at $t = 0$ and accumulates over time.

### Production Vector

The production vector is a variable that indicates how the space is occupied and how much time is remaining for the production of each good.

## Goods, Money and Prices

For the purpose of complete the market dynamics of the model, we need to make certain assumptions about the goods in order to facilitate price and money injection calculations.


### Goods

All goods are compose by two main characteristics: space per unit and time of production

#### Space per Unit

The space per unit of a good is the space occupied to produce 100 grams of the product. We will use 100 grams as a unit reference. Each good has a different space per unit constant and it doesnt depend on any other factors. 

#### Time of Production

At the same time, each product has a production time that depends on a gamma distribution $X \sim \Gamma(\alpha, \lambda)$
where the parameters $\alpha$ and $\lambda$ are unique for each good. 

### Money 

Each period of time there is a money injection based on the goods produced and the social utily value of each good, first lets define social utily value of each good $a_j$ $j = 1,2 \ldots, n$ for $r$ consumers with $k$ preferences and a preference matrix $M_{n \times k} $:

$$
SUV_{a_j} = \sum_{i = 1}^r \sum_{j = 1}^k  M_{n \times k}(a_j) 
$$

where $ M_{n \times k}(a_j)$ is an entry of the preference of each good $a_j$ in each basket of preference of each individual. With this and the production of each individual, we calculate the money injection $M_t$ of each period $t$, given $n$ types of goods where $P_i^{a_j}$ is the production of the good $a_j$ by the individual $i$: 

$$

M_t = \sum_{i = 1}^r \sum_{h = 1}^n SUV(a_j) \times P_i^{a_j}

$$


### Libraries


In [54]:
import pandas as pd
import numpy as np 
import random

### Creating Goods

In [57]:

# List of indoor-grown foods or vegetables in Spanish
food_names = [
    "tomate", "lechuga", "cebolla", "zanahoria", "pepino", "pimiento", "ajo",
    "fresa", "albahaca", "espinaca", "cilantro", "champiñón"
]

# List to store the dictionaries
food_data = []

# Function to calculate shape and scale parameters for gamma distribution
def calculate_parameters(mean):
    # Calculate the scale parameter (theta) based on the mean and interval
    sigma = 1  # Desired interval of +- 1 from the mean
    theta = mean / ((1.96 * sigma) ** 2)

    # Calculate the shape parameter (k) based on the mean and theta
    k = mean / theta

    return (k, theta)

# Create 12 dictionaries
for name in food_names:
    space = random.randint(0, 6)  # Random integer between 0 and 6
    mean = random.uniform(8, 10)  # Random mean between 8 and 10

    # Calculate shape and scale parameters
    shape, scale = calculate_parameters(mean)

    # Create the dictionary
    food_dict = {
        "nombre": name,
        "space": space,
        "gamma_parameters": (shape, scale)
    }

    food_data.append(food_dict)

# Print the list of dictionaries
for food in food_data:
    print(food)


{'nombre': 'tomate', 'space': 3, 'gamma_parameters': (3.8415999999999997, 2.0894941886736507)}
{'nombre': 'lechuga', 'space': 6, 'gamma_parameters': (3.8415999999999997, 2.2754655797090235)}
{'nombre': 'cebolla', 'space': 2, 'gamma_parameters': (3.8416, 2.1805865432621525)}
{'nombre': 'zanahoria', 'space': 3, 'gamma_parameters': (3.8415999999999992, 2.4553670009494253)}
{'nombre': 'pepino', 'space': 0, 'gamma_parameters': (3.8416, 2.1600462204395487)}
{'nombre': 'pimiento', 'space': 3, 'gamma_parameters': (3.8416, 2.1784636995068696)}
{'nombre': 'ajo', 'space': 2, 'gamma_parameters': (3.8415999999999997, 2.299935070719395)}
{'nombre': 'fresa', 'space': 6, 'gamma_parameters': (3.8416, 2.39370996251397)}
{'nombre': 'albahaca', 'space': 1, 'gamma_parameters': (3.8415999999999997, 2.4371680975816306)}
{'nombre': 'espinaca', 'space': 2, 'gamma_parameters': (3.8415999999999997, 2.1095218465301406)}
{'nombre': 'cilantro', 'space': 0, 'gamma_parameters': (3.8415999999999997, 2.098137252878605)

### Creating Individuals

In [53]:
class Individual :
    def __init__(self):
        self.dimension = 12
        self.space = np.random.randint(1, 20)
        self.balance = 0
        self.preferences_matrix = self.generate_preferences_matrix()
        self.suv_vector = self.generate_suv_vector()
        self.production_vector = np.zeros(self.dimension)

    def generate_preferences_matrix(self):
        n = self.dimension
        if n <= 0:
            raise ValueError("n must be a positive integer")

        matrix = np.random.rand(n, n)
        matrix /= matrix.sum(axis=1, keepdims=True)  # Ensure rows sum to 1
        matrix = np.round(matrix, 2)  # Round elements to two decimal places
        return matrix
    
    def generate_suv_vector(self):
        # Calculate the average of each column
        average_vector = np.mean(self.preferences_matrix, axis=0)
        return average_vector

    
individual = Individual ()
print(individual.production_vector)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
